In [1]:
from PandaStation import *
from PandaGrasping import *

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG
import subprocess


import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp,
    RandomGenerator, UniformlyRandomRotationMatrix
    )
import pydrake.perception as mut
import open3d as o3d
from ompl import base as ob
from ompl import geometric as og
import time
from enum import Enum

ycb = {"cracker": "drake/manipulation/models/ycb/sdf/003_cracker_box.sdf", 
    "sugar": "drake/manipulation/models/ycb/sdf/004_sugar_box.sdf", 
    "soup": "drake/manipulation/models/ycb/sdf/005_tomato_soup_can.sdf", 
    "mustard": "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf", 
    "gelatin": "drake/manipulation/models/ycb/sdf/009_gelatin_box.sdf", 
    "meat": "drake/manipulation/models/ycb/sdf/010_potted_meat_can.sdf",
    "brick": "drake/examples/manipulation_station/models/061_foam_brick.sdf"}

for key in ycb.keys():
    ycb[key] = FindResourceOrThrow(ycb[key])
    
ycb["puck"] = FindResource("models/puck.sdf")
ycb["marble"] = FindResource("models/marble.urdf")
ycb["dumbbell"] = FindResource("models/dumbbell.urdf")

def ycb_resource(name):
    global ycb
    return ycb[name]

def random_ycb_resource():
    global ycb
    ycb_items = list(ycb.items())
    index = np.random.randint(0, len(ycb_items))
    return ycb_resource(ycb_items[index][0])

In [46]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, 0.0)
parser = Parser(plant)
panda_hand = parser.AddModelFromFile(FindResource("models/welded_panda_hand.sdf"))
plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("panda_hand"))
#parser.AddModelFromFile(FindResource("models/panda_hand_fixed_collisions.urdf"))
#parser.AddModelFromFile(FindResourceOrThrow("drake/manipulation/models/wsg_50_description/sdf/schunk_wsg_50_with_tip.sdf"))


plant.Finalize()
meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=scene_graph.get_query_output_port(),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
meshcat.load()
diagram = builder.Build()
diagram_context = diagram.CreateDefaultContext()


frame = plant.GetFrameByName("panda_hand", panda_hand)
plant_context = plant.GetMyContextFromRoot(diagram_context)
#plant.SetPositions(plant_context, panda_hand, [0.04, 0.04])
print(frame.CalcPoseInWorld(plant_context))

diagram.Publish(diagram_context)

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6090...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7090/static/
Connected to meshcat-server.
RigidTransform(
  R=RotationMatrix([
    [1.0, 0.0, 0.0],
    [0.0, 1.0, 0.0],
    [0.0, 0.0, 1.0],
  ]),
  p=[0.0, 0.0, 0.0],
)
